In [ ]:
import warnings
warnings.filterwarnings('ignore')

import datasets
import chromadb
from sentence_transformers import SentenceTransformer

In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # msmarco-MiniLM-L-6-v3

In [3]:
dataset = datasets.load_dataset('wikipedia', '20220301.simple', trust_remote_code=True)
dataset = dataset['train']

In [4]:
chroma_client = chromadb.PersistentClient(path='wikipedia.db')
collection = chroma_client.get_or_create_collection('wikipedia')

In [5]:
num_docs = min(len(dataset), 1000)
batch_size = 128
ids = []
embeddings = []
for i in range(num_docs):
    ids.append(str(i))
    encoding = model.encode(dataset[i]['text'])
    embeddings.append(encoding.tolist())
    if (i + 1) % batch_size == 0 or i == num_docs - 1:
        collection.upsert(ids=ids, embeddings=embeddings)
        ids = []
        embeddings = []

In [6]:
query = 'run sports'
encoding = model.encode(query)
results = collection.query(query_embeddings=encoding.tolist(), n_results=5)
ids = [int(_id) for _id in results['ids'][0]]
titles = [dataset[_id]['title'] for _id in ids]
titles

['Sport',
 'Track and field athletics',
 'Long jump',
 'Association football',
 'Football']